## Teilautomatisierung der THG-Abschätzung nach GEST-Ansatz 

### Schritt 1: 

Offenland/Waldbiotope auf Torfböden zuschneiden und Offenland/Wald trennen.

### Schritt 2.1: 

QGIS-Tabelle der gewünschten Biotope exportieren 

### Schritt 2.2: 

Biotope mit Datenliste abgleichen und Arten rauskopieren --> automatisiert, Code siehe unten 

### Schritt 3.1: 

Arten den Wasserstufen "+" und "°" zuweisen 

### Schritt 3.2: 

"+" und "°" für jedes Biotop zählen und vorherrschende Wasserstufe(n) ermitteln 

### Schritt 3.3: 

Ergebnisse in einem neuen Sheet kompakt darstellen

### Schritt 4: 

Mit Fläche und GEST-Bezeichnung das CO2 bzw. den CO2-Äq-Wert pro Biotop berechnen (Ist-Zustand) 

### Schritt 5: 

Mit PrognoseTool-Ergebnissen die neuen Wasserstufen, Bezeichnungen und somit Emissionseinsparungen ableiten 


## Schritt 1: Offenland/Waldbiotope auf Torfböden zuschneiden und Offenland/Wald trennen.

Skript zum automatisierten Erstellen von Offenland/
Waldflächen, die sich auf Torfböden befinden. 

Geändert werden muss: 
Name des Biotop- und Torflayers, Spalte der Attribut-
Tabelle und Namen der dortigen Kategorien wie "Grünland", 
nach denen gefiltert werden soll (siehe expression im Code). 

Der folgende Skriptblock kann in der QGIS Python-Konsole ausgeführt werden. 

In [ ]:
# HIER bitte die Namen der Layer im GIS angeben 
biotope_name = "Biotope_GEST"
torfboeden_name = "Torfboeden"
output_name = "offenlandbiotope.shp"

# HIER Spalten- und Featurename ändern
expression = '"Hckurz" IN (\'Moor\', \'Grünland\')'

import sys
import os 

from qgis.core import (
    QgsProject, QgsVectorLayer, QgsFeatureRequest,
    QgsVectorFileWriter, QgsProcessingFeatureSourceDefinition
)

biotope_layer = QgsProject.instance().mapLayersByName(biotope_name)[0]
torfboeden_layer = QgsProject.instance().mapLayersByName(torfboeden_name)[0]



# expression = '"Hckurz" IN (\'Moor\', \'Grünland\', \'Gewässer\')'

# In den folgenden auskommentierten Zeilen kann (falls gewünscht) überprüft werden, wie viele Biotope extrahiert werden würden. 

# features = biotope_layer.getFeatures(QgsFeatureRequest().setFilterExpression(expression))
# count = sum(1 for f in features)
# print(f"Gefundene Features: {count}")

biotope_filtered = biotope_layer.materialize(QgsFeatureRequest().setFilterExpression(expression))

params = {
    'INPUT': biotope_filtered,
    'PREDICATE': [0],  # "intersects"
    'INTERSECT': torfboeden_layer,
    'METHOD': 0,       
    'OUTPUT': 'memory:'
}
result = processing.run("native:extractbylocation", params)
extracted_layer = result['OUTPUT']

torfboeden_path = torfboeden_layer.dataProvider().dataSourceUri().split("|")[0]
output_dir = os.path.dirname(torfboeden_path)

# HIER Name der Outputdatei ändern
output_path = os.path.join(output_dir, output_name)

QgsVectorFileWriter.writeAsVectorFormat(
    extracted_layer,
    output_path,
    "UTF-8",
    driverName="ESRI Shapefile"
)

print(f"Shapefile gespeichert unter: {output_path}")

## Schritt 2.1: QGIS-Tabelle der gewünschten Biotope exportieren

Für Ueckermünde ist ein automatisiertes Exportieren nicht sinnvoll, da die Biotope aller Teilgebiete in einem Layer sind, aber sollte dies z.B. für Hartmannsdorf gewünscht sein, einfach den folgenden Code ausführen. 

Vor dem Exportieren im Optimalfall schon die Größe jedes Biotops in einer neuen QGIS-Spalte der Attributtabelle berechnen ($area im Feldrechner), da diese in Schritt 4 gebraucht wird.

Auch nochmal überprüfen, ob wirklich nur Offenlandbiotope dabei sind, bspw. bei "Moor" können sich immer noch Waldabschnitte reinschleichen. 

An Bibliotheken werden wieder os sowie QgsVectorLayer und QgsVectorFileWriter aus Schritt 1 benötigt (falls getrennt bearbeitet, nochmal per import-Befehl einladen).

In [ ]:
# HIER den Pfad der exportierten .shp Datei aus Schritt 1 angeben. In einem zusammenhängenden Schritt kann auch der output_path aus Schritt 1 genommen 
# werden. Da die Schritte aber getrennt ausgeführt werden, ergab sich diese Struktur. 
shp_pfad = r""



Im folgenden Schritt wird der Vektorlayer eingeladen. Offenlandbiotope_GEST ist mein Beispielsname, hier kann irgendein String eingefügt werden. 
ogr ist der Provider, wie der Layer eingelesen wird, z.B. standardmäßig für .shp Dateien.

In [ ]:
layer = QgsVectorLayer(shp_pfad, "Offenlandbiotope_GEST", "ogr")


In [ ]:
gis_pfad = os.path.splitext(shp_pfad)[0] + ".xlsx"


QgsVectorFileWriter.writeAsVectorFormat(
    layer,
    gis_pfad,
    "UTF-8",
    driverName="XLSX",
    onlySelected=False
)

print(f"xlsx wurde exportiert nach: {gis_pfad}")

## Schritt 2.2: Biotope mit Datenliste abgleichen und Arten rauskopieren

Das folgende Skript soll dazu dienen, automatisch Biotope aus den DBU-Artenlisten auszulesen. 

In [1]:
# Diese Bibliotheken einfach einladen
import pandas as pd 

In [2]:
# HIER ist nochmal der Gis-Pfad aus Schritt 2.1 zu finden. Richtigen GIS-Pfad der exportierten Tabelle hier angeben! 
gis_pfad = r""

In [3]:
# HIER den Pfad der Artenliste, Biotope aus QGIS sowie gewünschten Output ändern wie gewünscht
dbu_artenliste_pfad = r""
output_pfad = r""

**WICHTIG**:
Die Artennummern in beiden Dateien müssen in der jeweiligen Excelzelle als ZAHL vorliegen, nicht als Text. Vor dem Ausführen nochmal in Excel überprüfen, 
sonst fehlen ggfs. Biotope.

In manchen Teilkartierungen der Ueckermünder Heide wurde gar keine FLID in die Datei eingetragen, allerdings mit FFH-Nummer in die BKFLID Spalte. Das bitte ebenfalls kontrollieren und ggfs. kurz manuell anpassen, sodass für jedes Biotop eine FLID vorhanden ist.

Der Excel-Befehl dafür ist =RECHTS(B2;LÄNGE(B2)-6) , die Zelle B2 an die gewünschte BKFLID-Zelle anpassen. 


In [4]:
artenliste_df = pd.read_excel(dbu_artenliste_pfad)
biotope_df = pd.read_excel(gis_pfad)

In [5]:
# HIER Biotop-Spalte der aus QGIS exportierten Datei auswählen, Python rechnet dabei immer Zahl = Spalte-1. 0 ist also Spalte 1, 1 ist Spalte 2 etc.
biotop_namen = biotope_df.iloc[:, 3]  
print(biotop_namen)

0     0410-334B4127
1     0410-334B4125
2     0410-334B4069
3     0410-334B4064
4     0410-334B4116
          ...      
86    0410-334B4042
87    0410-334B4092
88    0410-334B4039
89    0410-334B4043
90    0410-334B4107
Name: FLID, Length: 91, dtype: object


In [6]:
# HIER nun in der DBU-Artenliste die relevante Spalte auswählen, die auch in der GIS-Biotopliste für den Abgleich zu finden sein soll. 
artenliste_ergebnis = artenliste_df[artenliste_df.iloc[:, 2].isin(biotop_namen)]

print(artenliste_ergebnis)

       NELSID Naturerbefläche           FLID  Zählraum HC/NC Biotopcode Land  \
35360    8010          Bellin  0410-334B4018         7    HC             VRP   
35361    8010          Bellin  0410-334B4018         7    HC             VRP   
35362    8010          Bellin  0410-334B4018         7    HC             VRP   
35363    8010          Bellin  0410-334B4018         7    HC             VRP   
35364    8010          Bellin  0410-334B4018         7    HC             VRP   
...       ...             ...            ...       ...   ...             ...   
40827    8010          Bellin  0410-334B7067         8    HC             RHU   
40828    8010          Bellin  0410-334B7067         8    HC             RHU   
40829    8010          Bellin  0410-334B7067         8    HC             RHU   
40830    8010          Bellin  0410-334B7067         8    HC             RHU   
40831    8010          Bellin  0410-334B7067         8    HC             RHU   

       Anteil Biotop-code [%]          

In [7]:
# HIER gefilterte Artenliste in Excel exportieren 

artenliste_ergebnis.to_excel(output_pfad, index=False)
print("Artenliste der ausgewählten Biotope exportiert")

Artenliste der ausgewählten Biotope exportiert


In Excel kann in der ausgegebenen Artenliste mit =SUMME(1/ZÄHLENWENN(C2:C349;C2:C349)) überprüft werden, wie viele Biotope berücksichtigt wurden. Die Felder C2 und C349 an Start und Ende der FLIDs anpassen. Im Falle der Ueckermünder Heide fehlen leider häufig Biotope. 

## Schritt 3.1: Arten den Wasserstufen "+" und "°" zuweisen

Es wird auf eine "Master-Tabelle" zugegriffen, um den Arten die passenden Wasserstufen zuzuweisen. Arten, die nicht in der Liste aufgeführt sind, werden ausgegeben und müssen manuell nachgeschlagen bzw. direkt in die Mastertabelle hinzugefügt werden, damit zukünftige Projektgebiete schneller bearbeitet werden können. 

Zuerst muss die exportierte GEST-Tabelle aus Schritt 2.2 manuell ins richtige Format gebracht werden: 
1. Spalte FLID
2. Spalte Wiss. Name
3. Spalte Deutscher Name
4. Spalte Deckung
5. Spalte und weiter: Wasserstufen von 6+ bis 5 - sowie 1 (einfach an der ersten Zeile der Mastertabelle orientieren)

In [8]:
import warnings
import openpyxl

In [9]:
mastertabelle = pd.read_excel(r"...GEST_Mastertabelle.xlsx") # PFAD ANPASSEN!
# HIER das aktuelle GEST-Gebiet als xlsx einladen
gest_gebiet = pd.read_excel(r"...")

In [10]:
artenspalte_master = mastertabelle.iloc[:, 1]
artenspalte_gest = gest_gebiet.iloc[:, 1]
wasserstufen_master = mastertabelle.iloc[:, 4:19]

In die folgende Liste werden die Arten gespeichert, welche keine Übereinstimmung mit der Mastertabelle aufweisen konnten. 

In [11]:
nicht_gefunden = []

In [12]:
# HIER sollten die Arten aus dem untersuchten Projektgebiet angezeigt werden
print(artenspalte_gest)


0                  Alnus glutinosa
1                    Arctium lappa
2                     Carex remota
3                Chelidonium majus
4                Cirsium oleraceum
                   ...            
1898                  Prunus padus
1899    Rhytidiadelphus squarrosus
1900                 Rumex acetosa
1901              Rumex sanguineus
1902                 Urtica dioica
Name: Wiss. Name, Length: 1903, dtype: object


In [13]:
for i, art in enumerate(artenspalte_master):

    matching_indices = gest_gebiet.index[gest_gebiet.iloc[:, 1] == art].tolist()
    
    if matching_indices:
        for idx in matching_indices:
            gest_gebiet.iloc[idx, 4:19] = wasserstufen_master.iloc[i].values

    # nicht_gefunden = [art for art in artenspalte_gest if art not in set(artenspalte_master)]
    nicht_gefunden = sorted(set(artenspalte_gest) - set(artenspalte_master))

    warnings.simplefilter(action='ignore', category=FutureWarning)


C:\Users\Koch\AppData\Local\Temp\ipykernel_6596\1358694965.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '+' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gest_gebiet.iloc[idx, 4:19] = wasserstufen_master.iloc[i].values
C:\Users\Koch\AppData\Local\Temp\ipykernel_6596\1358694965.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '+' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gest_gebiet.iloc[idx, 4:19] = wasserstufen_master.iloc[i].values


**HIER** den Pfad ändern, wo die Resultate abgespeichert werden sollen (in meinem Fall dieselbe Datei). **IMMER AKTUALISIEREN**, damit keine alte Datei überschrieben wird.  

In [14]:
gest_gebiet.to_excel(r"...", index=False)



In [15]:
print(nicht_gefunden)


[]


**WICHTIG**: Die automatisierte Zuordnung der Wasserstufen sollte immer stichprobenartig überprüft werden. Zudem sollte für trockene Mineralböden (falls diese vorkommen) überprüft werden, ob die richtigen Wasserstufen ausgewählt wurden (verschiedene Tabellen/Übersichten als Quelle). 

Bei Bedarf können manuell noch die Baumarten aus der Tabelle entfernt oder in ein eigenes Sheet kopiert werden, das wurde noch nicht automatisiert.

## Schritt 3.2: "+" und "°" für jedes Biotop zählen und vorherrschende Wasserstufe(n) ermitteln 

Der nächste Schritt zählt alle eingetragenen Wasserstufen und gibt die Summen aus. Hierfür müssen die + und ° der Tab. 4-11, welche als Tendenz bei "Gleichstand" herangezogen werden, andere Zeichen bekommen. Für Ueckermünde habe ich "p" als Plus und "pklein" für das ° genommen. Welcher String letztendlich genommen wird, ist aber egal, da das Skript nur die "+"-Werte aufaddiert. 

**Vor dem Nutzen des folgenden Codes** muss bereits für jedes Biotop unter den jeweiligen Arten eine extra Zeile angelegt werden, in der lediglich in Spalte 1 die FLID steht, sonst nichts. (Also z.B. nach allen sortierten 0410-334B4013 eine letzte leere Zeile nur mit 0410-334B4013 ganz vorne). 
Leider noch nicht automatisiert, bei Bedarf lässt sich das in Zukunft auch noch anpassen. 



In [16]:
import openpyxl
from openpyxl.styles import Font, PatternFill
from collections import defaultdict

In [17]:
wasserstufen_pfad = r"..."

Es folgt die Funktion zum Zählen der Wasserstufen, einfach ausführen und nichts ändern. **Das Sheet mit den ausgelesenen Wasserstufen muss "WS_Zaehlen" heißen.** Es wird dann ein neues Sheet "WS_gezaehlt" mit den Ergebnissen angelegt.

### IMMER vorsichtshalber eine Kopie des bisherigen Dokuments anlegen, bevor das folgende Skript ausgeführt wird!

In [18]:
def wasserstufen_zaehlen(wasserstufen_pfad):

    wb = openpyxl.load_workbook(wasserstufen_pfad)

    if "WS_Zaehlen" not in wb.sheetnames:
        raise Exception("Worksheet 'WS_Zaehlen' nicht gefunden")

    ws_result = wb.create_sheet("WS_gezaehlt")
    ws_source = wb["WS_Zaehlen"]

    # Dunkelblaue Farbe für die Excel-Zeilen, damit man sie von den Zeilen der Arten unterscheiden kann. 
    fill_color = PatternFill(start_color="8EA9DB",
                           end_color="8EA9DB",
                           fill_type="solid")
    
    for row in ws_source.iter_rows():
        ws_result.append([cell.value for cell in row])
    
    # es werden jetzt alle einzigartigen werte gesucht, also alle FLIDs. 
    groups = defaultdict(list)
    for idx, row in enumerate(ws_result.iter_rows(min_row=2, max_col=1), start=2):
        if row[0].value is not None:
            groups[row[0].value].append(idx)
    
    for group_value, rows in groups.items():
        last_row = rows[-1]
        
        # Für jede Spalte E bis S (5 bis 19)
        for col in range(5, 20):
            plus_count = 0
            
            for row in rows:
                cell_value = ws_result.cell(row=row, column=col).value
                if cell_value == '+':
                    plus_count += 1
            
            ws_result.cell(row=last_row, column=col, value=plus_count)
            
            ws_result.cell(row=last_row, column=col).font = Font(bold=True)
            ws_result.cell(row=last_row, column=col).fill = fill_color
    
    wb.save(wasserstufen_pfad)
    print(f"'WS_gezaehlt' wurde in {wasserstufen_pfad} ergänzt.")

Nun die Funktion mit dem angegebenen Pfad ausführen, die dortige Datei wird überschrieben und mit den Ergebnissen aktualisiert.

In [20]:
wasserstufen_zaehlen(wasserstufen_pfad)

'WS_gezaehlt' wurde in M:\DBU Ueckermünder Heide\Planung\GEST\GEST_Test.xlsx ergänzt.


## Schritt 3.3: Wasserstufen-Ergebnisse in einem neuen Sheet kompakt darstellen 

Bei einer hohen Anzahl von Biotopen kann auch das Rüberkopieren der Ergebniszeilen lange dauern. Der folgende Codeblock erstellt ein neues Sheet "Ergebnis_Wasserstufen", die Spalte "Ergebnis" bleibt aber noch leer und wird manuell ausgefüllt, da dort auch der Standort sowie ggfs. die p / pklein-Werte miteinbezogen werden, die das Skript nicht erkennt. 

In [105]:
def wasserstufen_kopieren (wasserstufen_pfad):
    wb = openpyxl.load_workbook(wasserstufen_pfad)
    if "WS_gezaehlt" not in wb.sheetnames:
        raise Exception("Worksheet 'WS_gezaehlt' nicht gefunden")

    ws_result = wb.create_sheet("Ergebnis_Wasserstufen")
    ws_source = wb["WS_gezaehlt"]

    ws_result.append([
    "Biotop/FLID", "Ergebnis", "6+", "5+", "4+", "3+", "2+", "2+-", "4+2-", "3+2-",
    "2-", "2+3-", "3+-", "3-", "4-", "5-", "1"
    ])
    
    for cell in ws_result[1]:
        cell.font = Font(bold=True)
    
    fill_color = PatternFill(start_color="8EA9DB", end_color="8EA9DB", fill_type="solid")
    
    for cell in ws_result[1][2:17]:  
        cell.fill = fill_color
        
# FARBE überprüfen
    # for row in ws_source.iter_rows(min_row=2):
    #    for cell in row[4:19]:
     #       fill = cell.fill
      #      if fill is not None and fill.fill_type == "solid":
       #         print(f"Zelle {cell.coordinate} hat Farbe: {fill.fgColor.rgb}, Typ: {fill.fgColor.type}")


    for row in ws_source.iter_rows(min_row=2):
        if all(cell.fill.start_color.rgb in ["008EA9DB", "8EA9DB"] for cell in row[4:19]):
            neue_spalte = []
            neue_spalte.append(row[0].value)

            neue_spalte.append(None)

            for cell in row[4:19]:
                neue_spalte.append(cell.value)
            
            ws_result.append(neue_spalte)
        # else:
            # print("Keine Ergebniszelle.")

    wb.save(wasserstufen_pfad)
    print(f"'Ergebnis_Wasserstufen' wurde in {wasserstufen_pfad} ergänzt.")

Die folgende Zeile ausführen, um das Sheet "Ergebnis-Wasserstufen" zu erstellen. 

In [106]:
wasserstufen_kopieren(wasserstufen_pfad)

'Ergebnis_Wasserstufen' wurde in M:\DBU Ueckermünder Heide\Planung\GEST\GEST_SchSee_ergaenzung.xlsx ergänzt.


Nach dem Ausführen des Skripts muss nun also für jedes Biotop manuell die passendste(n) Wasserstufe(n) herausgesucht werden. 

## Schritt 4: Mit Fläche und GEST-Bezeichnung das CO2 bzw. den CO2-Äq.-Emissionswert pro Biotop berechnen (Ist-Zustand)

Das Erstellen der nächsten Tabelle "Ist-Zustand" wurde automatisiert und die Ergebnisse aus dem vorherigen Sheet rüberkopiert, ebenso wird aus dem in Schritt 2.1 exportierten File (z.B. SchSee_GIS) jetzt die Fläche sowie der DBU-Biotopname ausgelesen und in das neue Sheet eingefügt. **WICHTIG**: Nochmal überprüfen, ob die Datei aus Schritt 2.1 richtig nach FLID sortiert wurde, damit die Reihenfolge der Biotope identisch ist und die Daten auch korrekt zugeordnet werden. 

**Außerdem die FLID in die erste Spalte der GIS-Datei packen. Sonst funktioniert es nicht!**

Die anschließende Zuordnung von DBU-Biotopnamen zu GEST-Bezeichnung sowie das Eintragen der Emissionswerte erfolgt manuell.

In [52]:
# Falls die Schritte separat oder nicht vollständig durchgeführt werden, hier nochmal der gis_pfad (schon in 2.2 angegeben): 
gis_pfad = r"..."

In [107]:
def ist_zustand_sheet (wasserstufen_pfad):
    wb = openpyxl.load_workbook(wasserstufen_pfad)
    if "Ergebnis_Wasserstufen" not in wb.sheetnames:
        raise Exception("Worksheet 'Ergebnis_Wasserstufen' nicht gefunden")

    ws_result = wb.create_sheet("Ist-Zustand")
    ws_source = wb["Ergebnis_Wasserstufen"]

    wb_zusatz = openpyxl.load_workbook(gis_pfad, data_only=True)
    ws_zusatz = wb_zusatz.active 

    mapping = {}
    for row in ws_zusatz.iter_rows(min_row=2):
        flid_raw = row[0].value
        if flid_raw is None:
            continue
        flid = str(flid_raw).strip()
        area = row[132].value  
        # print(area)
        # print(type(area))
        biotoptyp = row[7].value 
        mapping[flid] = {"area": area, "biotoptyp": biotoptyp}


    ws_result.append([
        "Biotop/FLID", "Ergebnis", "Fläche [m2]", "Fläche [ha]", "Biotoptyp",
        "Bezeichnung (de) GEST", "EF [t CO2-Äq/ha*a]", "CO2 der Fläche [t CO2-Äq/a]"
    ])

    for cell in ws_result[1]:
        cell.font = Font(bold=True)
            
    for row in ws_source.iter_rows(min_row=2, values_only=True):
        flid_raw = row[0]
        if flid_raw is None:
            continue
        flid = str(flid_raw).strip()
        ergebnis = row[1]

        info = mapping.get(flid, {})
        area = info.get("area", "")
        area_ha = area / 10000 if isinstance(area, (int, float)) else ""
        biotoptyp = info.get("biotoptyp", "")

        ws_result.append([flid, ergebnis, area, area_ha, biotoptyp, "", "", ""])
        
    wb.save(wasserstufen_pfad)
    print(f"'Ist-Zustand' wurde in {wasserstufen_pfad} ergänzt.")

Die folgende Zeile ausführen, um das Sheet "Ist-Zustand" zu erstellen. 

In [108]:
ist_zustand_sheet(wasserstufen_pfad)

'Ist-Zustand' wurde in M:\DBU Ueckermünder Heide\Planung\GEST\GEST_SchSee_ergaenzung.xlsx ergänzt.


Schritt 5 wird Stand 01.08.25 noch manuell bearbeitet. 